# AUTOMATIC GIF OBJECT REMOVAL
Team: Abisha Fenn, Jackson Terwelp, Cassidy Petrykowski, Hunter Jensen, Kyle Koesters

Known issues: 
Colab will not process large GIFs through TRACER, 
because Google limits the amount of GPU power you can use on the free version of Colab. If you have the paid version, you will likely be okay.

In [ ]:
#@title Setup cell
% cd /content/ 
!rm -r ece301-lama-to-video

print('\n> Clone repo')
!git clone https://github.com/terwelja/ece301-lama-to-video

print('\n> Install dependencies')
!pip install -r ece301-lama-to-video/lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Download the big-lama model')
% cd /content/ece301-lama-to-video/lama/
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip

% cd /content/
print('\n> Fix opencv and torch')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet
!pip install torchtext==0.9.0 --quiet
!pip install torchvision==0.9.0 --quiet

print('\n> Init gif translation code')
import base64, os
from shutil import copyfile
import shutil
from PIL import Image, ImageSequence
import glob

# Upload your GIF here: (MAKE SURE TO RUN)
Have this cell expanded so that you can see the file upload button

In [38]:
% cd /content/
print('\n> USER INPUT: Import gif')
from google.colab import files
files = files.upload()
fname = list(files.keys())[0]

shutil.rmtree('./gif', ignore_errors=True)
! mkdir gif
copyfile(fname, f'./gif/{fname}')
os.rename(fname, 'input.gif')

% cd gif

print('\n> Converting gif into png sequence...')
if '.gif' in fname:
  #Thanks Hunter
  im = Image.open(fname)
  frame_duration = im.info['duration']

  index = 1
  for frame in ImageSequence.Iterator(im):
      frame.save("frame%d.png" % index)
      index += 1
  os.remove(fname)

else:
  print(f'Error: unknown suffix .{fname.split(".")[-1]} use [.gif]')

print('\n> ...DONE CREATING PNG SEQUENCE...')

/content

> USER INPUT: Import gif


Saving tornadogif2.gif to tornadogif2.gif
/content/gif

> Converting gif into png sequence...

> ...DONE CREATING PNG SEQUENCE...


# Disappear
Just hit run after uploading the gif above.
Check /content/output.gif and /content/output_mask.gif for the output gifs.

In [39]:
if os.path.isfile('output.gif'):
    os.remove('output.gif')
    os.remov('output_mask.gif')
    print("Old outputs removed successfully...")
 
else:
    print("No old outputs exist...")

print('\n> Copying pngs to TRACER...')
shutil.rmtree('/content/ece301-lama-to-video/TRACER/data/custom_dataset', ignore_errors=True)
! mkdir /content/ece301-lama-to-video/TRACER/data/custom_dataset
! cp -RT /content/gif/ /content/ece301-lama-to-video/TRACER/data/custom_dataset/

print('\n> Running TRACER...')
% cd /content/ece301-lama-to-video/TRACER 
shutil.rmtree('/content/ece301-lama-to-video/TRACER/pred_map/custom_dataset', ignore_errors=True)
! mkdir /content/ece301-lama-to-video/TRACER/pred_map/custom_dataset
! python main.py inference --dataset custom_dataset/ --arch 7 --img_size 640 --save_map True

print('\n> Creating and storing mask gif (/content/output.gif)...')

# filepaths
fp_in = "/content/ece301-lama-to-video/TRACER/pred_map/custom_dataset/*.png"
fp_out = "/content/ece301-lama-to-video/TRACER/pred_map/custom_dataset/image.gif"

# Ty Kyle
sort = sorted(sorted(glob.glob(fp_in)),key = len)
imgs = (Image.open(f) for f in sort)
img = next(imgs)  # extract first image from iterator
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=frame_duration, loop=0)

#Rename output
% cd /content/ece301-lama-to-video/TRACER/pred_map/custom_dataset
files = glob.glob('/content/ece301-lama-to-video/TRACER/pred_map/custom_dataset/*')

for f in files:
    ftitle, fext = os.path.splitext(f)
    os.rename(f, ftitle + '_mask' + fext)

shutil.copyfile('/content/ece301-lama-to-video/TRACER/pred_map/custom_dataset/image_mask.gif','/content/output_mask.gif')

print('\n> Copying TRACER output and input images to LaMa')
shutil.rmtree('/content/ece301-lama-to-video/lama/data_for_prediction', ignore_errors=True)
% cd /content/ece301-lama-to-video/TRACER/pred_map/custom_dataset
os.remove('image_mask.gif')
% cd /content/ece301-lama-to-video/lama
! mkdir data_for_prediction

! cp -RT /content/ece301-lama-to-video/TRACER/pred_map/custom_dataset/ /content/ece301-lama-to-video/lama/data_for_prediction/
! cp -RT /content/ece301-lama-to-video/TRACER/data/custom_dataset/ /content/ece301-lama-to-video/lama/data_for_prediction/

print('\n> Running LaMa...')
shutil.rmtree('/content/output', ignore_errors=True)
!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output  dataset.img_suffix=.png > /dev/null

#Rename files
files = glob.glob('/content/output/*')
for f in files:
    os.rename(f, f.replace('_mask', ''))

print('\n> Creating finished gif...')

# filepaths
fp_in = "/content/output/*.png"
fp_out = "/content/output/output.gif"

sort = sorted(sorted(glob.glob(fp_in)),key = len)
imgs = (Image.open(f) for f in sort)
img = next(imgs)  # extract first image from iterator
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=frame_duration, loop=0)

shutil.copyfile('/content/output/output.gif','/content/output.gif')

print('\n> ...FINISHED...')
print('\n> See output /content/output.gif')

from IPython.display import HTML

HTML('<img src="../output_mask.gif">')
HTML('<img src="../output.gif">')



> Copying pngs to TRACER...

> Running TRACER...
/content/ece301-lama-to-video/TRACER
<---- Training Params ---->
Namespace(RFB_aggregated_channel=[32, 64, 128], action='inference', arch='7', aug_ver=2, batch_size=32, channels=[24, 40, 112, 320], clipping=2, criterion='API', data_path='data/', dataset='custom_dataset/', denoise=0.93, epochs=100, exp_num=0, frequency_radius=16, gamma=0.1, img_size=640, lr=5e-05, lr_factor=0.1, model_path='results/', multi_gpu=True, num_workers=4, optimizer='Adam', patience=5, save_map=True, scheduler='Reduce', seed=42, weight_decay=0.0001)
<----- Initializing inference mode ----->
Loaded pretrained weights for efficientnet-b7
###### pre-trained Model restored #####
test length : 40
100% 2/2 [00:11<00:00,  5.60s/it]
time: 11.206s

> Creating and storing mask gif (/content/output.gif)...
/content/ece301-lama-to-video/TRACER/pred_map/custom_dataset

> Copying TRACER output and input images to LaMa
/content/ece301-lama-to-video/TRACER/pred_map/custom_datas